In [1]:
import logging
import multiprocessing
import threading
from tqdm import tqdm
import niapy
from niapy.algorithms.basic import (
    BatAlgorithm,
    FireflyAlgorithm,
    ParticleSwarmOptimization,
)
from niapy.algorithms import Algorithm
from niapy.problems import Problem
from niapy.problems.sphere import Sphere
from niapy.task import Task
import torch
from torch import nn
from torch.utils import data
import torchinfo
from PIL import Image
import pandas
import sklearn.model_selection
import sklearn.preprocessing
from matplotlib import pyplot as plt
import os
import numpy as np
from numpy.random import default_rng

from util.optimization_data import SingleRunData, PopulationData
from util.diversity_metrics import DiversityMetric
from util.constants import RNG_SEED, DATASET_PATH, BATCH_SIZE, EPOCHS, MAX_ITER, NUM_RUNS

execute_training = True

In [2]:
def optimization(algorithm: Algorithm, task: Task, single_run_data: SingleRunData):
    r"""An adaptation of NiaPy Algorithm run method.

    Args:
        algorithm (Algorithm): Algorithm.
        task (Task): Task with pre configured parameters.
        single_run_data (SingleRunData): Instance for archiving optimization results
    """
    try:
        algorithm.callbacks.before_run()
        algorithm.rng = default_rng(seed=RNG_SEED)
        pop, fpop, params = algorithm.init_population(task)
        # reset seed to random
        algorithm.rng = default_rng()
        xb, fxb = algorithm.get_best(pop, fpop)
        while not task.stopping_condition():
            algorithm.callbacks.before_iteration(pop, fpop, xb, fxb, **params)
            pop, fpop, xb, fxb, params = algorithm.run_iteration(
                task, pop, fpop, xb, fxb, **params
            )

            # save population data
            pop_data = PopulationData(
                population=np.array(pop), population_fitness=np.array(fpop)
            )
            pop_data.calculate_metrics(
                [
                    DiversityMetric.PDC,
                    DiversityMetric.PED,
                    DiversityMetric.PMD,
                    DiversityMetric.AAD,
                    DiversityMetric.PDI,
                ],
                task.problem,
            )
            single_run_data.add_population(pop_data)

            algorithm.callbacks.after_iteration(pop, fpop, xb, fxb, **params)
            task.next_iter()
        algorithm.callbacks.after_run()
        return xb, fxb * task.optimization_type.value
    except BaseException as e:
        if (
            threading.current_thread() is threading.main_thread()
            and multiprocessing.current_process().name == "MainProcess"
        ):
            raise e
        algorithm.exception = e
        return None, None

In [3]:
def optimization_runner(
    algorithm: Algorithm, problem: Problem, max_iter: int, runs: int
):
    r"""An adaptation of NiaPy ALgorithm run method.

    Args:
        algorithm (Algorithm): Algorithm.
        problem (Problem): Optimization problem.
        max_iter (int): Optimization stopping condition.
        runs (int): Number of runs to execute.
    """
    for r in tqdm(range(runs)):
        task = Task(problem, max_iters=max_iter)

        single_run_data = SingleRunData(
            algorithm_name=algorithm.Name,
            algorithm_parameters=algorithm.get_parameters(),
            problem_name=problem.name(),
            max_iters=max_iter,
        )

        results = optimization(algorithm, task, single_run_data)
        single_run_data.export_to_json(
            os.path.join(
                DATASET_PATH, algorithm.Name[0], problem.name(), f"run_{r:05d}.json"
            )
        )

In [4]:
for algorithm in [BatAlgorithm(), FireflyAlgorithm(), ParticleSwarmOptimization()]:
    sphere = Sphere()
    #optimization_runner(algorithm, sphere, MAX_ITER, NUM_RUNS)

### ML Data generation

In [5]:
class data_generator(torch.utils.data.Dataset):
    def __init__(self, data_path_list, classes) -> None:
        super().__init__()
        self.data_path_list = data_path_list
        self.classes = classes

    def __len__(self):
        return len(self.data_path_list)

    def __getitem__(self, index):
        run = SingleRunData.import_from_json(self.data_path_list[index])
        metrics = run.get_population_metrics()
        # normalize values
        sklearn.preprocessing.minmax_scale(metrics, feature_range=(0, 1), axis=0, copy=True)
        one_hot = np.zeros_like(self.classes, float)
        one_hot[self.classes.index(run.algorithm_name[0])] = 1

        return torch.tensor(metrics), torch.tensor(one_hot)

### Model

In [6]:
class LSTM(nn.Module):
    def __init__(self, input_len, num_classes, num_hidden=256, num_layers=3) -> None:
        super().__init__()

        self.lstm = nn.LSTM(
            input_size=input_len,
            hidden_size=num_hidden,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.8
        )
        self.fc = nn.Linear(num_hidden, num_classes)

    def forward(self, x):
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)
        features = hidden[-1]
        out = self.fc(features)

        return features, out

### Data loading and preprocessing

In [7]:
dataset_paths = []
classes = []
for algorithm in os.listdir(DATASET_PATH):
    classes.append(algorithm)
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        for run in os.listdir(os.path.join(DATASET_PATH, algorithm, problem)):
            dataset_paths.append(os.path.join(DATASET_PATH, algorithm, problem, run))

x_train, x_test = sklearn.model_selection.train_test_split(dataset_paths, test_size=0.2, shuffle=True, random_state=42)
x_train, x_val = sklearn.model_selection.train_test_split(x_train, test_size=0.25, shuffle=True, random_state=42)

In [ ]:
train_dataset = data_generator(x_train, classes)
train_data_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4)

val_dataset = data_generator(x_val, classes)
val_data_loader = data.DataLoader(val_dataset, batch_size=1, shuffle=True, pin_memory=True, num_workers=4)

test_dataset = data_generator(x_test, classes)
test_data_loader = data.DataLoader(test_dataset, batch_size=1, shuffle=True, pin_memory=True, num_workers=4)

input, target = next(iter(train_data_loader))
print(np.shape(input))
print(np.shape(target))

In [9]:
def nn_train(model, train_data_loader, val_data_loader, epochs, loss_fn, optimizer, device, model_file_name):
    loss_values = []
    val_loss_values = []

    for epoch in range(epochs):
        loss_sum = 0.0
        batch_count = 0
        val_loss_sum = 0.0
        val_batch_count = 0
            
        model.train()
        for batch in train_data_loader:
            input, target = batch
            
            target = target.to(device)
            input = input.to(device)

            _, pred = model(input)
            loss = loss_fn(pred, target.argmax(1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()
            batch_count += 1

        model.eval()
        with torch.no_grad():
            for batch in val_data_loader:
                input, target = batch
                
                target = target.to(device)
                input = input.to(device)

                _, pred = model(input)
                loss = loss_fn(pred, target.argmax(1))

                val_loss_sum += loss.item()
                val_batch_count += 1

        loss_values.append(loss_sum/batch_count)
        val_loss_values.append(val_loss_sum/val_batch_count)
        print(f"epoch: {epoch + 1}, loss: {loss_sum/batch_count :.10f}, val_loss: {val_loss_sum/val_batch_count :.10f}")

        torch.save(model, model_file_name)

    x = [*range(1, epochs+1)]
    plt.plot(x, loss_values, label="train loss")
    plt.plot(x, val_loss_values, label="val loss")
    plt.legend()
    plt.show()

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
model = LSTM(NUM_RUNS, len(classes))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()
model_file_name = f"lstm_model.pt"

print(torchinfo.summary(model, (100, 3), depth=5))

if execute_training:
    model.to(device)
    nn_train(model, train_data_loader, val_data_loader, EPOCHS, loss_fn, optimizer, device, model_file_name)
    torch.save(model, model_file_name)
else:
    model = torch.load(model_file_name, map_location=torch.device(device))
    model.to(device)
    loss_plot = np.asarray(Image.open('loss_plot.png'))
    plt.axis("off")
    plt.imshow(loss_plot)